In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Attempt 1: Get ADP data from myfantasyleague.com dev site
- +JSON is organized
- +Easily accessed
- -too much info only really need QB, RB, WR, TE, and D/ST

In [2]:
response = requests.get('http://football.myfantasyleague.com/2010/export?TYPE=adp&L=&W=&JSON=1')
data = response.json()
data = data['adp']['player']
adp=pd.DataFrame(data)
print adp.head()


  averagePick draftsSelectedIn     id maxPick minPick
0        3.35            12777  11192     201       1
1        5.32            13097  11201     203       1
2        5.41            13309   8658     191       1
3        5.73            13108   9988     172       1
4        6.00            13230   9054     187       1


In [3]:
response = requests.get('http://football.myfantasyleague.com/2010/export?TYPE=players&L=&W=&JSON=1')
data = response.json()
data = data['players']['player']
players=pd.DataFrame(data)
print players

     id                   name position status team
0  0151         Bills, Buffalo     TMWR    NaN  BUF
1  0152    Colts, Indianapolis     TMWR    NaN  IND
2  0153        Dolphins, Miami     TMWR    NaN  MIA
3  0154  Patriots, New England     TMWR    NaN  NEP
4  0155         Jets, New York     TMWR    NaN  NYJ


In [4]:
print players[players['id'] == adp['id'][0]]

         id           name position status team
1531  11192  Bell, Le'Veon       RB    NaN  PIT


## Get weekly performance stats starting with 2010
- only has top scorers. scrubs and bye week players do not show up
- easily scraped and cleaned


In [66]:
def get_DF_from_table(season, week):
    url = 'http://www.thehuddle.com/stats/%s/plays_weekly.php?week=%s&pos=qb&col=FPTS&ccs=1' % (season, week)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    if not soup:
        print season
        print week
        pass
    data = soup.table
    head = data.thead.find_all('th')[5:]

    headers = []
    for th in head:
        colname = th.a.text.strip()
        headers.append(colname)

    rows = data.tbody.find_all('tr')
    body = []
    for r in rows:
        cells = r.find_all('td')
        cur_row = []
        for c in cells:
            cell = c.text.strip()
            cur_row.append(cell)
        body.append(cur_row)
    df2 = pd.DataFrame(body, columns = headers)
    df2['SEASON'] = season
    df2['WEEK'] = week
    return df2

seasons = ['2010', '2011', '2012', '2013', '2014', '2015']
weeks = ['1','2', '3', '4', '5', '6', '7','8','9','10','11','12','13','14','15','16']
frames =[]
for s in seasons:
    for w in weeks:
        frames.append(get_DF_from_table(s, w))
        
result = pd.concat(frames)
print result.shape


(3536, 15)


In [68]:
print result.tail()

              PLAYER  NFL PLAYS FPTS RUN RYD RTD PASS CMP PYDS PTD FUM INT  \
37  Tarvaris Jackson  SEA     1   -0   1  -1   0    0   0    0   0   0   0   
38    Trevor Siemian  DEN     1   -0   1  -1   0    0   0    0   0   0   0   
39    Kellen Clemens   SD     1   -0   1  -1   0    0   0    0   0   0   0   
40      Chase Daniel   KC     2   -0   2  -2   0    0   0    0   0   0   0   
41      Drew Stanton  ARI     3   -0   2  -2   0    1   0    0   0   0   0   

   SEASON WEEK  
37   2015   15  
38   2015   15  
39   2015   15  
40   2015   15  
41   2015   15  
